In [1]:
import sys
import numpy as np
import pandas as pd

In [17]:
(13466-265) + (21555-13467) + (25384-21562) + (26220-25392) + \
(26472-26244) + (27387-27201) + (27759-27393) 

27085

In [2]:
import time

In [3]:
sys.path.append('../')

In [4]:
import bjorn_support as bs
import onion_trees as ot
import mutations as bm
import visualize as bv
import reports as br

In [5]:
subs = pd.read_csv('/home/al/analysis/gisaid/subs_long_2021-01-20.csv.gz', compression='gzip')
dels = pd.read_csv('/home/al/analysis/gisaid/dels_long_2021-01-20.csv.gz', compression='gzip')

In [6]:
print(subs.shape)
print(dels.shape)
subs['type'] = 'substitution'
muts = pd.concat([subs, dels])
print(muts.shape)

(5946054, 38)
(103463, 44)
(6049517, 47)


In [7]:
muts.columns

Index(['idx', 'replacements', 'pos', 'gene', 'codon_num', 'ref_codon',
       'alt_codon', 'ref_aa', 'alt_aa', 'mutation', 'strain', 'virus',
       'gisaid_epi_isl', 'genbank_accession', 'date', 'region', 'country',
       'division', 'location', 'region_exposure', 'country_exposure',
       'division_exposure', 'segment', 'length', 'host', 'age', 'sex',
       'Nextstrain_clade', 'pangolin_lineage', 'GISAID_clade',
       'originating_lab', 'submitting_lab', 'authors', 'url', 'title',
       'paper_url', 'date_submitted', 'purpose_of_sequencing', 'type',
       'seq_len', 'del_positions', 'del_len', 'relative_coords',
       'absolute_coords', 'del_seq', 'prev_5nts', 'next_5nts'],
      dtype='object')

In [ ]:
meta_info = ['strain', 'date','date_submitted',
        'country', 'division', 'location', 
        'submitting_lab', 'originating_lab',
        'authors', 'pangolin_lineage', 'GISAID_clade', 
        'gisaid_epi_isl', 'genbank_accession',
        'purpose_of_sequencing']

muts_info = ['type', 'mutation', 'gene', 
             'ref_codon', 'pos', 'alt_codon', 
             'is_synonymous', 
             'ref_aa', 'codon_num', 'alt_aa', 
             'absolute_coords', 
             'del_len']

In [ ]:
collection_date
submitted_date
name
original_lab
submitting_lab
gisaid_epi_isl
genbank_accession
country
division
location
pangolin_lineage
gisaid_clade
additional info: Eg, screened by S gene dropout for b117
other_info
Mutations -> mutations
Mut,
gene
2. ref_codon
3. alt_codon
4. ref_aa
5. alt_aa
6. is_synonymous: boolean
7. pos_aa
8. pos_nuc
9. aa_map_coords: (Cumulative length of genes in aa)
9. nuc_map_coords: (Cumulative length of genes in aa)
10. orf1ab_pos_aa
11. orf1ab_pos_nuc
12. orf1ab_gene_name
13. mutation_type: ["insertion", "substitution", "deletion"]
14. length

In [12]:
sample_ids = muts[['strain']].drop_duplicates().sample(10)['strain'].unique()

In [13]:
test = muts[muts['strain'].isin(sample_ids)]

In [8]:
muts['is_synonymous'] = False
muts.loc[muts['ref_aa']==muts['alt_aa'], 'is_synonymous'] = True

In [16]:
start = time.time()
(test.groupby(['strain', 'country', 'division', 'location',
               'date','date_submitted', 'authors', 
               'pangolin_lineage', 'GISAID_clade', 'gisaid_epi_isl',
               'submitting_lab', 'originating_lab',
               'purpose_of_sequencing'], as_index=True)
             .apply(lambda x: x[['type', 'mutation', 'gene', 'pos', 'absolute_coords', 
                                 'del_len', 'ref_codon', 'alt_codon', 'is_synonymous',
                                 'ref_aa', 'codon_num', 'alt_aa']].to_dict('records'))
             .reset_index()
             .rename(columns={0:'Mutations'})
             .to_json('test_data_model_2021-01-21.json', orient='records'))
end = time.time()
print(f'Execution time: {end - start} seconds')

Execution time: 0.060778141021728516 seconds


In [17]:
muts.shape

(6049517, 48)

In [11]:
print(f'Execution time: {(end - start)/60} minutes')

Execution time: 11.696144835154216 minutes


In [14]:
gbycols = ['date', 'country', 'division', 'location', 
           'pangolin_lineage', 'authors',  'submitting_lab',
           'originating_lab', 'strain']
ans = subs.groupby(gbycols).agg(mutations=('mutation', 'unique')).reset_index()

In [15]:
mois = ['ORF1a:I4205V', 'ORF1b:D1183Y', 'S:S13I', 'S:W152C', 'S:L452R']
ans['is_vui'] = ans['mutations'].apply(bv.is_vui, args=(set(mois),))

In [16]:
ans.loc[(ans['is_vui']==True) & (ans['date']=='2020-07-14')]

,date,country,division,location,pangolin_lineage,authors,submitting_lab,originating_lab,strain,mutations,is_vui
152316,2020-07-14,United States of America,California,Los Angeles County,B.1.324,P. Hemarajata et al. et al,Los Angeles County PHL,USC Clinical Lab,USA/CA-LACPHL-AE00059/2020,"[5UTR:R81C, ORF1a:T265I, ORF1a:N310S, ORF1a:V7...",True


In [17]:
# data = subs.loc[(subs['mutation']=='S:L452R')&(subs['location'].str.contains('San Diego'))]


# Data Priming

In [18]:
is_test = False
msa_fp = '/home/al/analysis/gisaid/sequences_2021-01-20_16-44_aligned.fasta.gz'
meta_fp = '/home/al/analysis/gisaid/metadata_2021-01-20_17-49.tsv.gz'
date = msa_fp.split('_')[1]

In [19]:
print(date)
print(msa_fp)
print(meta_fp)

2021-01-20
/home/al/analysis/gisaid/sequences_2021-01-20_16-44_aligned.fasta.gz
/home/al/analysis/gisaid/metadata_2021-01-20_17-49.tsv.gz


In [ ]:
subs, _ = bm.identify_replacements_per_sample(msa_fp, meta_fp,  
                                           bm.GENE2POS, data_src='gisaid', 
                                           is_gzip=True, test=is_test)

Loading Alignment file at: /home/al/analysis/gisaid/sequences_2021-01-20_16-44_aligned.fasta.gz
Initial cleaning...
Identifying mutations...


In [ ]:
subs_fp = f'/home/al/analysis/gisaid/subs_long_{date}.csv.gz'
subs.to_csv(subs_fp, index=False, compression='gzip')

In [ ]:
dels, _ = bm.identify_deletions_per_sample(msa_fp, meta_fp,  
                                           bm.GENE2POS, data_src='gisaid', 
                                           min_del_len=3, is_gzip=True, test=is_test)

In [ ]:
dels_fp = f'/home/al/analysis/gisaid/dels_long_{date}.csv.gz'
dels.to_csv(dels_fp, index=False, compression='gzip')

In [9]:
(dels.groupby(['mutation', 'absolute_coords', 'del_len', 'del_seq'])
     .agg(num_samples=('idx', 'nunique'))
     .reset_index()
     .nlargest(50, 'num_samples'))

,mutation,absolute_coords,del_len,del_seq,num_samples
4,S:DEL69/70.0,21765:21770,6,catgtc,9
1,ORF1a:DEL3675/3677.0,11288:11296,9,tggttttaa,7
3,S:DEL144/144.0,21991:21993,3,att,7
0,ORF1a:DEL141/143.0,686:694,9,gtcatttga,1
2,ORF7a:DEL79/80.0,27626:27631,6,gccaga,1


# Analysis Report Generation

In [4]:
# feature = 'mutations'
# values = ['S:S13I', 'S:W152C', 'S:L452R']
feature = 'mutation'
# S:Q677H, M:A85S, N:D377Y,
values = ['ORF1a:I4205V', 'ORF1b:D1183Y', 'S:S13I', 'S:W152C', 'S:L452R']#, 'S:L452R'] # 'ORF1ab:I4205V', 
input_params = {
    'gisaid_data_fp' : '/home/al/analysis/gisaid/subs_long_2021-01-18.csv.gz',
    'gisaid_meta_fp': '/home/al/analysis/gisaid/metadata_2021-01-18_18-48.tsv.gz',
    'vocs': ['B.1.1.7', 'B.1.1.70'],
    'strain': 'CAVUI1',
    'date': '01/21/2021',
    'msa_fp': Path('/home/al/analysis/gisaid/sequences_2021-01-18_08-48_aligned.fasta.gz'),
    'meta_fp' : Path('/home/al/code/HCoV-19-Genomics/metadata.csv'),
    'tree_fp' : Path('/home/al/analysis/alab_mutations_01-01-2021/alab/seqs_aligned.fa.treefile'),
    'subs_fp' : '/home/al/analysis/alab_mutations_01-01-2021/alab_substitutions_long_01-01-2021.csv',
    'countries_fp' : '/home/al/data/geojsons/countries.geo.json',
    'states_fp' : "/home/al/data/geojsons/us-states.json",
    'counties_fp' : '/home/al/data/geojsons/us-counties.json',
    'patient_zero' : 'NC_045512.2',
    'gisaid_data_fp' : '/home/al/analysis/gisaid/subs_long_2021-01-18.csv.gz',
    'gisaid_meta_fp': '/home/al/analysis/gisaid/metadata_2021-01-18_18-48.tsv.gz',
    'b117_meta' : '/home/al/analysis/b117/nextstrain_groups_neherlab_ncov_S.N501_metadata.tsv',
    'b117_tree': 'test_data/b117_seqs_aligned.fasta.treefile',
    'sample_sz': 300,
    'sampling_img_fp' : "/home/al/analysis/b117/figs/sars-cov-2_EM_v3.png"
}

In [5]:
results = br.generate_voc_data(feature, values, input_params)

In [6]:
html = br.generate_voc_html(feature, values, results, template_name='mut.html')
# br.save_html(html, f'test_data/orf1ab_i4205v_report.html')
br.save_html(html, f'test_data/cavui1_report.html')

In [7]:
# feature = 'mutations'
# values = ['S:S13I', 'S:W152C', 'S:L452R']
feature = 'mutation'
# S:Q677H, M:A85S, N:D377Y,
values = ['S:S13I', 'S:W152C', 'S:L452R']#, 'S:W152C', 'S:L452R'] # 'ORF1ab:I4205V', 
input_params['strain'] = 'CAVUI1S'

In [8]:
results = br.generate_voc_data(feature, values, input_params)

In [9]:
html = br.generate_voc_html(feature, values, results, template_name='mut.html')
# br.save_html(html, f'test_data/orf1ab_i4205v_report.html')
br.save_html(html, f'test_data/cavui1s_report.html')

In [12]:
runs = {'orf1a_i4205v': 'ORF1a:I4205V', 'orf1b_d1183y': 'ORF1b:D1183Y', 
        's_s13i': 'S:S13I', 's_w152c': 'S:W152C', 's_l452r': 'S:L452R'}
for name, mut in runs.items():
    if mut=='S:W152C':
        input_params['strain'] = mut
        results = br.generate_voc_data(feature, [mut], input_params)
        html = br.generate_voc_html(feature, [mut], results, template_name='mut.html')
        br.save_html(html, f'test_data/{name}_report.html')

In [17]:
gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')

In [24]:
mutations = ['S:Q677H', 'M:A85S', 'N:D377Y']

In [30]:
m = gisaid.groupby(['country', 'division', 'strain']).agg(mutations=('mutation', 'unique')).reset_index()


In [31]:
m['is_vui'] = m['mutations'].apply(bv.is_vui, args=(set(mutations),))

In [32]:
m.loc[m['is_vui']==True, 'division'].value_counts()

Ontario          25
New York         10
Maryland          2
Wisconsin         2
West Virginia     1
Massachusetts     1
Michigan          1
Texas             1
Name: division, dtype: int64

In [16]:
# res = (gisaid.groupby(['date', 'country', 'division', 
#                                         'purpose_of_sequencing',
#                                         'location', 'pangolin_lineage', 'strain'])
#                        .agg(mutations=('mutation', 'unique')).reset_index())
# res['is_vui'] = res['mutations'].apply(bv.is_vui, args=(set(values),))

In [12]:

def get_mutations(data: pd.DataFrame, lineage: str='B.1.1.7'):
    mutations = set(data[data['pangolin_lineage']==lineage]['mutation'].unique().tolist())
    return mutations